## Subgraphs
- Shared schema keys
- Different schemas

In [3]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, END, add_messages, START
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode

In [57]:
load_dotenv()

class SubState(TypedDict):
    messages: Annotated[list, add_messages]
# Substate as child state
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

In [58]:
def agent(state: SubState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

def router(state: SubState):
    prev_message = state["messages"][-1]

    if(hasattr(prev_message,"tool_calls") and len(prev_message.tool_calls) > 0):
        return "tool"
    else:
        return "end"

In [59]:
tool_node = ToolNode(tools)

subgraph = StateGraph(SubState)

subgraph.add_node("agent", agent)
subgraph.add_node("tool_node", tool_node)

subgraph.add_edge(START, "agent")
subgraph.add_conditional_edges("agent", router, {"tool":"tool_node", "end": END})
subgraph.add_edge("tool_node", "agent")

child_app = subgraph.compile()

In [61]:
child_app.invoke({"messages": [HumanMessage(content="How is today's weather in Vaasa, Finland?")]})

{'messages': [HumanMessage(content="How is today's weather in Vaasa, Finland?", additional_kwargs={}, response_metadata={}, id='64823e68-e604-4ec0-9f91-2b5309e8d173'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Vaasa, Finland today"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--06e06657-61e1-4d8a-904a-e2f0497ddb09-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Vaasa, Finland today'}, 'id': '2b15d488-ac23-449f-baad-28226606401f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 16, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"title": "Vaasa January 2025 Historical Weather Data (Finland)", "url": "https://weatherspark.com/h/m/87765/2025/1/Historical-Weather-in-

### 1) Multi-agent with parent and child graph
- Shared schema keys

In [46]:
from typing import List
# Solution: add child_graph as a node in the parent graph


class ParentState(TypedDict):
    messages: Annotated[List, add_messages]

parent_graph = StateGraph(ParentState)

# Adding compiled child_graph-child_app as a node to parent_graph
parent_graph.add_node("search_agent", child_app)

parent_graph.add_edge(START, "search_agent")
parent_graph.add_edge("search_agent", END)

parent_app = parent_graph.compile()

outcome = parent_app.invoke({"messages": [HumanMessage(content="How is today's weather in Vaasa, Finland ")]})
# outcome = parent_app.stream({"messages": [HumanMessage(content="How is the weather in Vaasa, Finland today")]}, stream_mode="values")
outcome["messages"][-1]
# outcome


AIMessage(content="I'm sorry, I don't have the current weather conditions for Vaasa, Finland. The search results give historical weather data for January 2025.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--84b44268-94a0-4036-9c1b-704dd2aaf227-0', usage_metadata={'input_tokens': 661, 'output_tokens': 36, 'total_tokens': 697, 'input_token_details': {'cache_read': 0}})

In [47]:
outcome
# for chunck in outcome:
#     print(chunck["messages"][-1].pretty_print())

{'messages': [HumanMessage(content="How is today's weather in Vaasa, Finland ", additional_kwargs={}, response_metadata={}, id='31676712-d35a-4a7f-aa19-6a71ca499087'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Vaasa, Finland today"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--7ff1a5d8-28c0-4eb8-987e-c5f9d661be0e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Vaasa, Finland today'}, 'id': '58154dac-82b7-4eb9-8a72-8c03c46374d7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 16, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"title": "Vaasa January 2025 Historical Weather Data (Finland)", "url": "https://weatherspark.com/h/m/87765/2025/1/Historical-Weather-in-

### 2) Multi-agent with parent and child graph
- Different schemas

In [64]:
from typing import Dict
# Solution- Invoke childgraph/subgraph in a node inside the parentgraph

class QueryState(TypedDict):
    query:str
    response: str

def search_agent1(state: QueryState) -> Dict:
    # Transformation from parent schema to subgraph schema
    subgraph_input = {"messages": [HumanMessage(content=state["query"])] }

    subgraph_result = child_app.invoke(subgraph_input)

    message = subgraph_result["messages"][-1]
  
    return {"response": message.content}


In [65]:
parent_graph = StateGraph(QueryState)

parent_graph.add_node("search_agent1", search_agent1)

parent_graph.add_edge(START, "search_agent1")
parent_graph.add_edge("search_agent1", END)

parent_app = parent_graph.compile()
parent_app.invoke(input={"query": [HumanMessage(content="How is today's weather in Vaasa, Finland ")]})

ValidationError: 3 validation errors for HumanMessage
content.str
  Input should be a valid string [type=string_type, input_value=[HumanMessage(content="Ho..., response_metadata={})], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
content.list[union[str,dict[any,any]]].0.str
  Input should be a valid string [type=string_type, input_value=HumanMessage(content="How...}, response_metadata={}), input_type=HumanMessage]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
content.list[union[str,dict[any,any]]].0.dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value=HumanMessage(content="How...}, response_metadata={}), input_type=HumanMessage]
    For further information visit https://errors.pydantic.dev/2.11/v/dict_type